In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# LU - разложение

Это представление матрицы в виде $A = L U$, где $L$ - нижняя треугольная матрица, $U$ - верхняя треугольная матрица. LU-разложение существует только в том случае, когда матрица $A$ обратима, а все ведущие (угловые) главные миноры матрицы $A$ невырождены.

Алгоритм решения:

1) Вначале надо найти разложение матрицы $A = a_{ij}$ на $L = l_{ij}$ и $U = u_{ij}$:

    $u_{1j} = a_{1j} \text{ , } j = 1,..., n$

    $l_{j1} = \dfrac{a_{j1}}{u_{11}} \text{ , } j = 2, ..., n$

    Если $i = 2, ..., n$:

    $u_{ij} = a_{ij} - \sum_{k = 1}^{i} l_{ik} u_{kj} \text{ , } j = i, ..., n$

    $l_{ji} = \dfrac{1}{u_{ii}} (a_{ji} - \sum_{k = 1}^{i} l_{jk} u_{ki}) \text{ , } j = i+1, ..., n$

2) Затем задача решеается в 2 этапа:

    $\bullet$ $L U x = y \Rightarrow U x = L^{-1} y$

    $\bullet$ $U x = L^{-1} y \Rightarrow x = U^{-1} L^{-1} y$ 

In [4]:
def LU_decomposition(A):
    n = len(A)
    L = np.zeros((n, n))
    U = np.zeros((n, n))
    
    for i in range(n):
        # Upper Triangular
        for k in range(i, n):
            sum = 0
            for j in range(i):
                sum += (L[i][j] * U[j][k])
            U[i][k] = A[i][k] - sum
        
        # Lower Triangular
        for k in range(i, n):
            if (i == k):
                L[i][i] = 1
            else:
                sum = 0
                for j in range(i):
                    sum += (L[k][j] * U[j][i])
                L[k][i] = (A[k][i] - sum) / U[i][i]
    
    return L, U

def L_solver(L, y):
    n = len(y)
    x = np.zeros(n)
    for i in range(n):
        sum = 0
        for j in range(i):
            sum += L[i][j] * x[j]        
        x[i] = 1/L[i, i] * (y - sum)
    return x

def U_solver(U, y):
    n = len(y)
    x = np.zeros(n)
    for i in range(n-1, -1, -1):
        sum = 0
        for j in range(n-1, i, -1):
            sum += U[i][j] * x[j]        
        x[i] = 1/U[i, i] * (y - sum)
    return x    

def LU_solver(A, y):
    lower, upper = LU_decomposition(A)
    return U_solver(upper, L_solver(lower, y))
    

# Example usage
matrix = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
lower, upper = LU_decomposition(matrix)

print("Lower Triangular Matrix:")
print(lower)

print("Upper Triangular Matrix:")
print(upper)

Lower Triangular Matrix:
[[1. 0. 0.]
 [4. 1. 0.]
 [7. 2. 1.]]
Upper Triangular Matrix:
[[ 1.  2.  3.]
 [ 0. -3. -6.]
 [ 0.  0.  0.]]


# Метод прогонки

Работает для 3 - диагональной матрицы, для которой выполнено условие диагонального преобладания. 

$$a_{m} x_{m - 1} + b_{m} x_{m} + c_{m} x_{m + 1}= y_{m} \text{ , } m \in 1, ..., n$$

Алгоритм решения:

1) Сначала ищем $P_{1} \text{ и } Q_{1}$ по формулам: $P_{1}=-\dfrac{c_{1}}{b_{1}}, \quad Q_{1}=\dfrac{y_{1}}{b_{1}}$

2) Затем ищем остальные $P_{m}$, $Q_{m}$ по формулам: 

    $P_{m+1}=-\dfrac{c_{m}}{b_{m}+a_{m} P_{m}}, \quad Q_{m+1}=\dfrac{y_{m} - a_{m} Q_{m}}{b_{m}+a_{m} P_{m}}$

3) Затем ищем $x_{n}$ по формуле: $x_{n} = \dfrac{y_{n} - a_{n} Q_{n}}{b_{n}+a_{n} P_{n}}$

4) Решение восстанавливается при помощи прогоночного соотношения: 

    $x_{m} = P_{m+1} x_{m+1} + Q_{m+1}$

In [5]:
def tridiagonal_matrix_algorithm(a, b, c, y):
    N = len(y)
    P = np.zeros(N-1)
    Q = np.zeros(N-1)
    x = np.zeros(N)    
    P[0] = - c[0]/b[0]
    Q[0] =  y[0]/b[0]
    for m in range(N-2):
        P[m+1] = - c[m]/(b[m] + a[m]*P[m])
        Q[m+1] = (y[m] - a[m]*Q[m])/(b[m] + a[m]*P[m])
    x[-1] = (y[-1] - a[-1]*Q[-1])/(b[-1] + a[-1]*P[-1])
    for m in range(N-2,-1,-1):
        x[m] = P[m]*x[m+1]+Q[m]
    return x

def tridiagonal_matrix_decomposition(A):
    N = np.shape(A)[0]
    a = np.zeros(N - 1)
    b = np.zeros(N)
    c = np.zeros(N - 1)
    for i in range(N-1):
        a[i] = A[i+1, i]
        b[i] = A[i, i]
        c[i] = A[i, i+1]
    b[N] = A[N, N]
    return a, b, c

def tridiagonal_matrix_solver(A, y):
    a, b, c = tridiagonal_matrix_decomposition(A)
    return tridiagonal_matrix_algorithm(a, b, c, y)


# Разложение Холецкого

Это представление симметричной положительно определённой матрицы $A$ в виде $A = L L^{T}$, где $L$ - нижняя треугольная матрица со строго положительными элементами на диагонали. Разложение Холецкого всегда существует и единственно для любой симметричной положительно определённой матрицы. По своей сути оно представляет $L U$ разложение симметричной матрицы, поэтому и алгоритм решения совпадает с таковым из 1 пункта.

In [6]:
def Cholesky_decomposition(A):
    n = len(A)
    L = np.zeros((n, n))
    
    for i in range(n):
        for j in range(i+1):
            if j == i:
                L[i][j] = np.sqrt(A[i][j] - np.sum(L[i][:j]*L[j][:j]))
            else:
                L[i][j] = (A[i][j] - np.sum(L[i][:j]*L[j][:j])) / L[j][j]
    return L

def L_solver(L, y):
    n = len(y)
    x = np.zeros(n)
    for i in range(n):
        sum = 0
        for j in range(i):
            sum += L[i][j] * x[j]        
        x[i] = 1/L[i, i] * (y - sum)
    return x

def U_solver(U, y):
    n = len(y)
    x = np.zeros(n)
    for i in range(n-1, -1, -1):
        sum = 0
        for j in range(n-1, i, -1):
            sum += U[i][j] * x[j]        
        x[i] = 1/U[i, i] * (y - sum)
    return x  

def Cholesky_solver(A, y):
    L = Cholesky_decomposition(A)
    return U_solver(np.transpose(L), L_solver(L, y))

# QR - разложение

$A = Q R$, где $Q$ - ортогональная матрица, а $R$ - верхнетреугольная матрица.

Алгоритм:

1) Сначала ортогонализируем столбцы матрицы $A$ при помощи алгоритма ортогонализации Грама - Шмидта, полученные векторы и будут составлять матрицу $Q$;

2) Далее находим верхнетреугольную матрицу $R = Q^{-1} A = Q^{T} A$

3) Решение получается в 2 этапа:

    $\bullet$ $Q R x = y \Rightarrow R x = Q^{-1} y = Q^{T} y$

    $\bullet$ $R x = Q^{T} y \Rightarrow x = R^{-1} Q^{T} y$ 

In [7]:
def QR_decomposition(A):
    m, n = A.shape
    Q = np.eye(m)
    R = np.copy(A)

    for j in range(n):
        # Вычисляем отражение Хаусхолдера для j-й колонки матрицы R
        normx = np.linalg.norm(R[j:, j])
        if R[j, j] < 0:
            normx = -normx

        u = R[j:, j] - normx * np.eye(len(R[j:, j]))[:, 0]
        v = u / np.linalg.norm(u)

        # Применяем отражение Хаусхолдера к матрицам Q и R
        R[j:, :] -= 2 * np.outer(v, np.dot(v, R[j:, :]))
        Q[:, j:] -= 2 * np.dot(Q[:, j:], np.outer(v, v))

    return Q.T, R